In [ ]:
library(DESeq2)
library('tximport') 
library('stringr') 
library('goseq') 
library('GOplot') 
library(lattice)
library('reshape')
library('dplyr')
library(ggplot2)
library('repr') 
library('svglite') 

In [ ]:
count.NG <- read.csv('/home/minsoo1012/Girdling_prj_revision/counting/NG_rep1/RSEM.genes.results', header = TRUE, sep='\t')
names(count.NG)<-c('Gene', 'transcript', 'length', 'eff_length', 'count', 'TPM', 'FPKM')
head(count.NG)
data.length<-count.NG[c('Gene', 'length')]
head(data.length)

In [ ]:
wd='/home/minsoo1012/Girdling_prj_revision/counting'
sample<-c('NG_rep1', 'G_rep1', 'G_rep2', 'G_rep3')
files<-file.path(wd,paste0(sample, '/RSEM.genes.results'))
names(files) <- sample

RSEM_input <- tximport(files, type = 'rsem', txOut = TRUE)
summary(RSEM_input)
head(RSEM_input$counts)

#import annotation
trinotate <- read.csv('/home/minsoo1012/Girdling_prj_revision/trinotate/Pr_trinotate.tsv',sep = '\t', header = TRUE)
head(trinotate)

In [ ]:
#set group names and replicates
condition<-factor(c('non_girdling', rep('girdling', 3)))
sampleTable <- data.frame(condition=condition)
rownames(sampleTable) <- colnames(RSEM_input$counts)

#run DESeq for count normalization and comparison
dds <- DESeqDataSetFromTximport(RSEM_input, sampleTable, ~condition)
dds <- DESeq(dds)
colnames(dds) <- condition

In [ ]:
#extract result from DESeq output 
res <- results(dds, contrast=c('condition', 'girdling', 'non_girdling'))  #FC=formal/latter=girdling/non_girdling
resDF <- as.data.frame(res)

resdata <- merge(as.data.frame(res), as.data.frame(counts(dds, normalized=TRUE)), by='row.names', sort=FALSE)
colnames(resdata)<-c('Gene','baseMean','log2FoldChange','lfcSE','stat','pvalue','padj','non_girdling','girdling','girdling.1','girdling.2')
resdata<-merge(resdata, trinotate, by='Gene')
resdata<-merge(resdata, data.length, by='Gene')
resdata<-resdata[!duplicated(resdata[c('Gene')]), ]
head(resdata)

In [ ]:
#make list of Gene : GOs to be used for GO analysis
list.GO=list()
for(i in c(1:nrow(resdata))){
    df.temp = resdata[i,]
    temp.GO = unlist(strsplit(df.temp$gene_ontology_BLASTX, split='`'))
    temp.GO=substr(temp.GO, 1,10)

    if(length(temp.GO) > 0){list.GO[[i]] = temp.GO}
    else{list.GO[[i]] = NA}
}
names(list.GO)=as.character(resdata$Gene)

In [ ]:
#make DEG-flag vector (1: Differentially expressed, 0: not significant)
resdata$DEG.flag = ( (resdata$padj < 0.05) & (abs(resdata$log2FoldChange) >= 1) )
vector.DEGenes = as.vector(as.numeric(resdata$DEG.flag))
names(vector.DEGenes) = resdata$Gene
vector.DEGenes[is.na(vector.DEGenes)] = 0 # make NA (zero-counts) to FALSE
seq.length=resdata$length
names(seq.length) = resdata$Gene

In [ ]:
GO.wall = goseq(pvalue.GO, gene2cat = list.GO, method = "Wallenius")
head(GO.wall)

In [ ]:
#melted version of GO list  -> required for nex cell: GO plot
header=c('Gene', 'GO')
df.temp.GO=read.csv('/home/minsoo1012/Girdling_prj_revision/DEanalysis/Pr_custom_GO.txt', sep='=', header=FALSE, colClasses=c("character", "character"))
names(df.temp.GO)=header
df.temp.GO$GO <- paste('GO',':', str_sub(df.temp.GO$GO,2,), sep = "")
head(df.temp.GO)

In [ ]:
row.names(GO.wall) = GO.wall$category
df.GOplot = data.frame( category = character(), ID = character(), term = character(), count = numeric(), genes = character(), logFC = numeric(), adj_pval = numeric(), zscore = numeric())
rownames(resdata) = resdata$Gene

for(j in 1:nrow(GO.wall)){
    #### get SeqName subset lists from list.GO.seq
    ##for each GO in the list of enriched GOs, retrieve list of genes containing that GO 
    list.GO.seq = subset(df.temp.GO, GO == GO.wall$category[j])
    list.GO.seq  <- list.GO.seq[!duplicated(list.GO.seq[c('Gene')]),]
    list.GO.seq = str_sub(list.GO.seq$Gene,,-2)
    ####C-1-3-2. get expression data from subset for those genes
    subset.expr.data = resdata[list.GO.seq, ]
    ####na.rm (removed genes with zero counts)
    subset.expr.data = subset.expr.data[!is.na(subset.expr.data$Gene),]
    ####remove genes that failed to calculate padj (due to single replicate)
    subset.expr.data = subset.expr.data[!is.na(subset.expr.data$padj),]

    ####up and down mark
    ####        current criteria: FDR < 0.1 & |logFC| > 1 
    #if nrow(subset.expr.data) == 0 --> skip
    if(nrow(subset.expr.data)>0){
        no.upregulated = sum((subset.expr.data$padj < 0.1) & (subset.expr.data$log2FoldChange > 1))
        no.downregulated = sum((subset.expr.data$padj < 0.1) & (subset.expr.data$log2FoldChange < -1))

        temp.res.enrich.GO = GO.wall[j,]

        df.temp.GOplot = data.frame(category = temp.res.enrich.GO$ontology,
        ID = temp.res.enrich.GO$category,
        term = temp.res.enrich.GO$term,
        count = temp.res.enrich.GO$numDEInCat,
        genes = subset.expr.data$Gene,
        logFC = subset.expr.data$log2FoldChange,
        adj_pval = temp.res.enrich.GO$over_represented_pvalue,
        zscore = ((no.upregulated - no.downregulated)/sqrt(nrow(subset.expr.data)))
        )

        df.GOplot = rbind(df.GOplot, df.temp.GOplot)
    }
}


In [ ]:

circ = subset(df.GOplot, adj_pval < 0.05)
circ_filtered = circ[with(circ, order(adj_pval)), ]
reduced_circ = reduce_overlap(circ_filtered)
GOBar(reduced_circ, display = "multiple")